<a href="https://colab.research.google.com/github/aditipanchal121/ML/blob/master/Shakespeare_Playwright_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import warnings
import os
import tensorflow as tf
import numpy as np

warnings.filterwarnings('ignore')

In [41]:
path = tf.keras.utils.get_file('shakespeare.txt',
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')

In [42]:
print("Length of text {} characters" .format(len(text)))
print(text[:250])


Length of text 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [43]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [44]:
char2idx = {u:idx for idx, u in enumerate(vocab)}
idx2char = np.array(vocab)
textAsInt = np.array([char2idx[char] for char in text])

In [45]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print("    {:4s}: {:3d}".format(repr(char), char2idx[char]))
print("}\n....\n")

print('{}  ------> characters mapped to int -----> {}'.format(repr(text[:13]),
                                                             textAsInt[:13]))

{
    '\n':   0
    ' ' :   1
    '!' :   2
    '$' :   3
    '&' :   4
    "'" :   5
    ',' :   6
    '-' :   7
    '.' :   8
    '3' :   9
    ':' :  10
    ';' :  11
    '?' :  12
    'A' :  13
    'B' :  14
    'C' :  15
    'D' :  16
    'E' :  17
    'F' :  18
    'G' :  19
}
....

'First Citizen'  ------> characters mapped to int -----> [18 47 56 57 58  1 15 47 58 47 64 43 52]


# Readying Data

In [46]:
seqLength = 100
examplesPerEpoch = len(text) // (seqLength + 1)

charDataset = tf.data.Dataset.from_tensor_slices(textAsInt)

In [47]:
for i in charDataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [48]:
sequences = charDataset.batch(seqLength + 1, drop_remainder=True)

In [49]:
for item in sequences.take(5):
    print(repr("".join(idx2char[item.numpy()])))
    

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [50]:
def splitInputTarget(chunk):
    inputText = chunk[:-1]
    targetText = chunk[1:]
    return inputText, targetText

In [51]:
dataset = sequences.map(splitInputTarget)

In [52]:
for inputEx, targetEx in dataset.take(1):
    print("Input data: ", repr("".join(idx2char[inputEx.numpy()])))
    print("Target data: ", repr("".join(idx2char[targetEx.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [53]:
for i, (inputIdx, targetIdx) in enumerate(zip(inputEx[:5], targetEx[:5])):
    print("step {:4d}".format(i))
    print("  input {} ({:s})".format(inputIdx, repr(idx2char[inputIdx])))
    print("  expected output {} ({:s})".format(targetIdx, repr(idx2char[targetIdx])))
    
    

step    0
  input 18 ('F')
  expected output 47 ('i')
step    1
  input 47 ('i')
  expected output 56 ('r')
step    2
  input 56 ('r')
  expected output 57 ('s')
step    3
  input 57 ('s')
  expected output 58 ('t')
step    4
  input 58 ('t')
  expected output 1 (' ')


Creating Training Sets

In [54]:
batchSize = 64
bufSize = 10000

dataset = dataset.shuffle(bufSize).batch(batchSize, drop_remainder=True)

print(dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [55]:
vocabSize = len(vocab)
embeddingDim = 256
rnnUnits = 1024

# BUILDING MODEL

In [56]:
def buildModel(vocabSize, embeddingDim, rnnUnits, batchSize):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocabSize, embeddingDim, batch_input_shape=[batchSize, None]),
        tf.keras.layers.GRU(rnnUnits, return_sequences=True, stateful=True,
                           recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocabSize)
    ])
    
    return model

In [57]:
model = buildModel(vocabSize=len(vocab), embeddingDim =embeddingDim, rnnUnits=rnnUnits, batchSize=batchSize)

In [58]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [60]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices)

[64 18 41  5 58 36 60 53 20  2 13 52 20 20 12 25  4 13 19 60 27 29 45 41
 33 46 60  2 51 47 49 22 13 47 13  8 11 53 36 54 36 17 20 12 47 37 26  7
 27 64 14 49 42 38  1 14  8  0  1  9 58 21  5 61 40 14 23 42  6 57  4 47
 25 43 15  0 56 62  7 61 59  7 25 51 63 47 20 45 26 20 56 50 49 57 45 47
 19 49 58 51]


In [61]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "ure keeps in grief:\nTherefore, no dancing, girl; some other sport.\n\nLady:\nMadam, we'll tell tales.\n\n"

Next Char Predictions: 
 "zFc'tXvoH!AnHH?M&AGvOQgcUhv!mikJAiA.;oXpXEH?iYN-OzBkdZ B.\n 3tI'wbBKd,s&iMeC\nrx-wu-MmyiHgNHrlksgiGktm"


In [62]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, 
                                                           logits, 
                                                           from_logits=True)

In [63]:
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1759076


# TRAINING MODEL

In [64]:
model.compile(optimizer="adam", loss=loss)

In [65]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [66]:
EPOCHS = 50

In [67]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 22s 127ms/step - loss: 2.6427
Epoch 2/50
172/172 [==============================] - 22s 126ms/step - loss: 1.9442
Epoch 3/50
172/172 [==============================] - 22s 127ms/step - loss: 1.6787
Epoch 4/50
172/172 [==============================] - 22s 127ms/step - loss: 1.5340
Epoch 5/50
172/172 [==============================] - 22s 127ms/step - loss: 1.4488
Epoch 6/50
172/172 [==============================] - 22s 127ms/step - loss: 1.3895
Epoch 7/50
172/172 [==============================] - 22s 127ms/step - loss: 1.3459
Epoch 8/50
172/172 [==============================] - 22s 127ms/step - loss: 1.3078
Epoch 9/50
172/172 [==============================] - 22s 127ms/step - loss: 1.2736
Epoch 10/50
172/172 [==============================] - 22s 126ms/step - loss: 1.2405
Epoch 11/50
172/172 [==============================] - 22s 126ms/step - loss: 1.2076
Epoch 12/50
172/172 [==============================] - 22s 126ms/step - lo

# PREDICTION

In [68]:
model = buildModel(vocabSize, embeddingDim, rnnUnits, batchSize=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [69]:
def generateText(model, startString):
    numGen = 500
    inputEval = [char2idx[s] for s in startString]
    inputEval = tf.expand_dims(inputEval, 0)
    
    textGen = []
    temperature = 1.0
    
    model.reset_states()
    
    for i in range(numGen):
        predictions = model(inputEval)
        predictions = tf.squeeze(predictions, 0)
        predictions /= temperature
        predictedId = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        inputEval = tf.expand_dims([predictedId], 0)
        textGen.append(idx2char[predictedId])
        
    return (startString + "".join(textGen))

In [70]:
print(generateText(model, startString ="HAMLET: "))

HAMLET: marry, Peace! O pity hath been
A book of prayer, be but testimonied
Are plebeians I'll kiss the city,
Or never bear this mercy if you fled to pieces.
Will you are welcome but he's so perfidion: but hadst thou there?

LUCIO:
This is an hour nothing.

MERCUTIO:
And for our reasons are tupoffent.

GLOUCESTER:

KING EDWARD IV:
Welcome, my lord; I am a rage
To bear the advantage thou twint on court'sied Henry lived:
Seld of imprisonments: therefore, noble lord,
Nor turns the devout religious mouther 
